<a href="https://colab.research.google.com/github/satheesh3180/load_breast_cancer-/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================
# Recurrent Neural Networks (RNNs) & LSTMs

# 1.Why Standard Neural Networks Fail on Sequences
# 2.Architecture and Flow
# 3.Training RNNs and Vanishing Gradient Problem
# ======================================================

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, SimpleRNN, LSTM, GRU, Bidirectional, Dropout
from tensorflow.keras.preprocessing import sequence
import numpy as np, random


# ---------------------------
# Data Loading and Preprocessing
# ---------------------------
num_words = 10000
maxlen = 200
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
word_index = imdb.get_word_index()


def encode(text):
    return sequence.pad_sequences([[word_index.get(w, 2) for w in text.lower().split()]], maxlen=maxlen)

# --------------------------------------------------------
# Session 1️⃣ - Why Standard Neural Networks Fail on Sequences
# --------------------------------------------------------
print("\n🧩Standard Neural Network (No memory between words)")
model_dense = Sequential([
    Embedding(num_words, 32, input_length=maxlen),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dense.fit(X_train, y_train, epochs=1, batch_size=256, validation_split=0.1, verbose=1)

samples = ["The movie was fantastic", "The movie was not fantastic"]
for text in samples:
    pred = model_dense.predict(encode(text))[0][0]
    print(f"Review: {text} | Predicted: {'Positive 😀' if pred>0.5 else 'Negative 😞'}")

print("👉 This model fails to understand 'not fantastic' because it treats words separately (no sequence memory).")

# --------------------------------------------------------
# Session 2️⃣ - How RNNs Work: Architecture and Flow
# --------------------------------------------------------
print("\n🧩SimpleRNN remembers previous words (sequential flow)")
model_rnn = Sequential([
    Embedding(num_words, 32, input_length=maxlen),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(X_train, y_train, epochs=1, batch_size=256, validation_split=0.1, verbose=1)

samples = ["The movie is not good", "The movie was fantastic,good movie"]
for text in samples:
    pred = model_rnn.predict(encode(text))[0][0]
    print(f"Review: {text} | Predicted: {'Positive 😀' if pred>0.5 else 'Negative 😞'}")

print("👉 RNN slightly understands order, but struggles with long sentences (memory fades).")

# --------------------------------------------------------
# Session 3️⃣ - Training RNNs and Vanishing Gradient Problem
# --------------------------------------------------------
print("\n🧩Vanishing Gradient (Deep SimpleRNN)")
model_vanish = Sequential([
    Embedding(num_words, 32, input_length=maxlen),
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])
model_vanish.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_vanish.fit(X_train, y_train, epochs=1, batch_size=512, validation_split=0.1, verbose=1)

text = "The movie started great but ended boring"
pred = model_vanish.predict(encode(text))[0][0]
print(f"Review: {text} | Predicted: {'Positive 😀' if pred>0.5 else 'Negative 😞'}")
print("👉 Deep RNN loses gradient → struggles to capture early info (vanishing gradient).")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🧩Standard Neural Network (No memory between words)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.5929 - loss: 0.6525 - val_accuracy: 0.8332 - val_loss: 0.3709
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Review: The movie was fantastic | Predicted: Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Review: The movie was not fantastic | Predicted: Negative 😞
👉 This model fails to understand 'not fantastic' because it treats words separately (no sequence memory).

🧩SimpleRNN remembers previous words (sequential flow)
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 100ms/step - accuracy: 0.5328 - loss: 0.6895 - val_accuracy: 0.6336 - val_loss: 0.6422
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Review: The movie is not good | Predicted: Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Review: The movie was fantastic,good movie | Predicted: Negative 😞
👉 RNN slightly understands order, but struggles with long sentences (memory fades).

🧩Vanishing Gradient (Deep SimpleRNN)
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 459ms/step - accuracy: 0.5006 - loss: 0.7002 - val_accuracy: 0